## Amazon Bedrock Guardrails: 
Amazon Bedrock Guardrails provides robust content filtering for generative AI applications. This customizable framework helps enterprises implement safety protocols for their AI applications by setting up filters to prevent harmful, inaccurate, or sensitive content. Guardrails enables control over topics, content appropriateness, and PII handling through configurable policies. Organizations can define what information should be blocked, filtered, or anonymized based on specific requirements. Built with regulatory compliance in mind, Guardrails allows developers to establish boundaries that keep AI interactions safe while maintaining functionality and utility. It integrates seamlessly with Bedrock's foundation models, providing protection without sacrificing performance or user experience.

in this notebook: we will perform the following 

## Basic Setup
- **UUID Generation**: Creates a unique client request token to ensure idempotency (prevents accidental duplicate guardrail creation)
- **Name and Description**: Sets clear identifiers for the guardrail's purpose ("AdvancedRagWorkshopGuardrails" focused on SEC filing content)
- **Blocked Messages**: Customized messages displayed when input or output is blocked by guardrails

## Topic Policy Configuration
- **Financial Advice (DENY)**: Blocks any content that recommends investments or financial decisions
- **Legal Interpretation (DENY)**: Prevents the model from interpreting legal or regulatory requirements
- **Future Performance**: There's a commented-out policy that would block speculation beyond disclosed forward-looking statements

## Content Policy Configuration
Sets high-strength filters for both input and output on:
- **HATE**: Blocks hateful content
- **INSULTS**: Filters insulting language
- **SEXUAL**: Prevents sexual content
- **VIOLENCE**: Blocks violent content
- **MISCONDUCT**: Filters content promoting illegal or harmful activities
- **PROMPT_ATTACK**: Only filters input (HIGH) but not output (NONE) for prompt injection attempts

## Contextual Grounding Policy
Sets thresholds for ensuring responses remain grounded to source material:
- **GROUNDING**: 0.1 threshold requires responses to be based on the provided context/documents
- **RELEVANCE**: 0.1 threshold ensures responses are relevant to the user's query

## Word Policy Configuration
Blocks specific words deemed inappropriate for the application:

## Sensitive Information Policy
### PII Entities
Configures ANONYMIZE action (redacts/masks content) for a comprehensive list of personal identifiable information types:
- Personal identifiers (NAME, EMAIL, PHONE, ADDRESS, AGE)
- Financial data (CREDIT_DEBIT_CARD details, BANK ACCOUNT numbers)
- Government IDs (SOCIAL_SECURITY_NUMBER, DRIVER_ID, PASSPORT_NUMBER)
- Technical identifiers (IP_ADDRESS, MAC_ADDRESS)
- Authentication data (PASSWORD, PIN, USERNAME)
- And many other PII types

### Regex Patterns
Custom regex patterns to catch domain-specific sensitive information:
- **Medical Record Numbers**: Pattern matching "123-456-7890" format
- **Insurance Numbers**: Pattern matching two letters followed by 7 digits

## Tags
Metadata for organizing and tracking the guardrail:
- **Environment**: "Production"
- **Department**: "Finance"

## Response Handling
Captures and outputs the:
- **Guardrail ID**: Unique identifier for the created guardrail
- **Guardrail ARN**: Full Amazon Resource Name
- **Version**: Version number of the created guardrail

## 1. Import Required Libraries

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError, BotoCoreError

# Initialize the Amazon Bedrock client in the us-west-2 region
bedrock = boto3.client('bedrock', region_name='us-west-2')


## 2. Load Configuration Variables

In [ ]:
with open("../variables.json", "r") as f:
    variables = json.load(f)

variables

## 3. Create a Guardrail

In [ ]:
import boto3
import uuid
import botocore.exceptions

# Initialize the Bedrock client
bedrock = boto3.client('bedrock', region_name='us-west-2')  # Adjust region as needed

def create_guardrail_if_not_exists(guardrail_name="AdvancedRagWorkshopGuardrails"):
    """
    Create a guardrail only if it doesn't already exist by name.
    Uses list_guardrails to check for existing guardrails with the same name.
    
    Parameters:
    - guardrail_name (str): The name of the guardrail
    
    Returns:
    - str: The guardrail ID if successful, None otherwise
    """
    # First, check if a guardrail with this name already exists
    try:
        print(f"Checking if guardrail '{guardrail_name}' already exists...")
        
        # List all guardrails
        response = bedrock.list_guardrails()
        
        # The API might return guardrails under 'guardrailSummaries' or 'guardrails' key
        guardrails_key = None
        if 'guardrailSummaries' in response:
            guardrails_key = 'guardrailSummaries'
        elif 'guardrails' in response:
            guardrails_key = 'guardrails'
        
        if not guardrails_key:
            print(f"Unexpected API response format. Keys: {list(response.keys())}")
            # If we can't find any expected key, assume no guardrails exist
            guardrails = []
        else:
            print(f"Found guardrails under key: {guardrails_key}")
            guardrails = response[guardrails_key]
            
            # Handle pagination if needed
            while 'nextToken' in response:
                response = bedrock.list_guardrails(nextToken=response['nextToken'])
                if guardrails_key in response:
                    guardrails.extend(response[guardrails_key])
            
            # Debug information
            print(f"Found {len(guardrails)} existing guardrails")
            
            # Check if our guardrail name already exists
            for guardrail in guardrails:
                # Check different possible field names for the name
                guardrail_name_fields = ['name', 'Name']
                for field in guardrail_name_fields:
                    if field in guardrail and guardrail[field] == guardrail_name:
                        # Try different possible field names for the ID
                        for id_field in ['id', 'guardrailId', 'Id', 'GuardrailId']:
                            if id_field in guardrail:
                                guardrail_id = guardrail[id_field]
                                print(f"Guardrail '{guardrail_name}' already exists with ID: {guardrail_id}")
                                print(f"Full guardrail record: {guardrail}")
                                return guardrail_id
                        
                        # If we found the name but not the ID, print the entire record
                        print(f"Found guardrail with matching name but couldn't identify ID field")
                        print(f"Guardrail record: {guardrail}")
                        return None
            
        print(f"No existing guardrail found with name '{guardrail_name}'. Proceeding to create...")
    
    except botocore.exceptions.ClientError as e:
        print(f"Error checking existing guardrails: {e}")
        print("Will attempt to create a new guardrail anyway.")
    
    # If we get here, the guardrail doesn't exist or we couldn't check
    # Generate a unique client request token for each request
    client_request_token = str(uuid.uuid4())
    
    try:
        # Create the guardrail
        response = bedrock.create_guardrail(
            name=guardrail_name,
            description="Restrict responses to AWS services and Amazon 10K filings content only",
            blockedInputMessaging="I can only process questions related to AWS services and Amazon 10K filings.",
            blockedOutputsMessaging="I'm an AWS specialist focused on Amazon's 10K filings. I can only provide information related to AWS services and Amazon's financial reporting.",
            topicPolicyConfig={
                'topicsConfig': [
                    {
                        'name': 'non-aws-topics',
                        'definition': 'Any questions or topics not related to AWS services, Amazon cloud infrastructure, or Amazon 10K financial reporting.',
                        'type': 'DENY'
                    },
                    {
                        'name': 'general-trivia',
                        'definition': 'General knowledge questions about geography, capitals, populations, history, or other non-AWS related facts.',
                        'type': 'DENY'
                    },
                    {
                        'name': 'financial-advice',
                        'definition': 'Any recommendations about investments or financial decisions',
                        'type': 'DENY'
                    },
                    {
                        'name': 'legal-interpretation',
                        'definition': 'Interpretation of legal or regulatory requirements',
                        'type': 'DENY'
                    }
                ]
            },
            contentPolicyConfig={
                'filtersConfig': [
                    {'type': 'HATE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'INSULTS', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'SEXUAL', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'MISCONDUCT', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'PROMPT_ATTACK', 'inputStrength': 'HIGH', 'outputStrength': 'NONE'}
                ]
            },
            contextualGroundingPolicyConfig={
                'filtersConfig': [
                    {'type': 'GROUNDING', 'threshold': 0.1},
                    {'type': 'RELEVANCE', 'threshold': 0.1}
                ]
            },
            wordPolicyConfig={
                'wordsConfig': [
                    {'text': 'material weakness'},
                    {'text': 'undisclosed liabilities'},
                    {'text': 'shareholder lawsuit'},
                    {'text': 'SEC investigation'},
                    {'text': 'accounting irregularities'},
                    {'text': 'restate earnings'},
                    {'text': 'liquidity crisis'},
                    {'text': 'bankruptcy risk'},
                    {'text': 'fraudulent activity'},
                    {'text': 'insider trading'}
                    
                ]
            },
            sensitiveInformationPolicyConfig={
                'piiEntitiesConfig': [
                    {'type': 'NAME', 'action': 'ANONYMIZE'},
                    {'type': 'EMAIL', 'action': 'ANONYMIZE'},
                    {'type': 'PHONE', 'action': 'ANONYMIZE'},
                    {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'ADDRESS', 'action': 'ANONYMIZE'},
                    {'type': 'AGE', 'action': 'ANONYMIZE'},
                    {'type': 'AWS_ACCESS_KEY', 'action': 'ANONYMIZE'},
                    {'type': 'AWS_SECRET_KEY', 'action': 'ANONYMIZE'},
                    {'type': 'CA_HEALTH_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'CREDIT_DEBIT_CARD_CVV', 'action': 'ANONYMIZE'},
                    {'type': 'CREDIT_DEBIT_CARD_EXPIRY', 'action': 'ANONYMIZE'},
                    {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'DRIVER_ID', 'action': 'ANONYMIZE'},
                    {'type': 'INTERNATIONAL_BANK_ACCOUNT_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'IP_ADDRESS', 'action': 'ANONYMIZE'},
                    {'type': 'LICENSE_PLATE', 'action': 'ANONYMIZE'},
                    {'type': 'MAC_ADDRESS', 'action': 'ANONYMIZE'},
                    {'type': 'PASSWORD', 'action': 'ANONYMIZE'},
                    {'type': 'PIN', 'action': 'ANONYMIZE'},
                    {'type': 'SWIFT_CODE', 'action': 'ANONYMIZE'},
                    {'type': 'UK_NATIONAL_HEALTH_SERVICE_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'UK_UNIQUE_TAXPAYER_REFERENCE_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'URL', 'action': 'ANONYMIZE'},
                    {'type': 'USERNAME', 'action': 'ANONYMIZE'},
                    {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'US_INDIVIDUAL_TAX_IDENTIFICATION_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'US_PASSPORT_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'VEHICLE_IDENTIFICATION_NUMBER', 'action': 'ANONYMIZE'},
                    {'type': 'US_BANK_ROUTING_NUMBER', 'action': 'ANONYMIZE'}
                ],
                'regexesConfig': [
                    {
                        'name': 'stock_ticker_with_price',
                        'description': 'Stock ticker with price pattern',
                        'pattern': '\\b[A-Z]{1,5}\\s*[@:]\\s*\\$?\\d+(\\.\\d{1,2})?\\b',
                        'action': 'ANONYMIZE'
                    },
                    {
                        'name': 'financial_figures',
                        'description': 'Large financial figures in billions/millions',
                        'pattern': '\\$\\s*\\d+(\\.\\d+)?\\s*(billion|million|B|M)\\b',
                        'action': 'ANONYMIZE'
                    },
                    {
                        'name': 'earnings_per_share',
                        'description': 'EPS figures',
                        'pattern': 'EPS\\s*(of)?\\s*\\$?\\d+\\.\\d{2}',
                        'action': 'ANONYMIZE'
                    },
                    {
                        'name': 'investor_relations_contact',
                        'description': 'Investor relations contact information',
                        'pattern': '(?i)investor\\s*relations\\s*[^\\n]+\\d{3}[\\.-]\\d{3}[\\.-]\\d{4}',
                        'action': 'ANONYMIZE'
                    }
                ]
            },
            tags=[
                {'key': 'Environment', 'value': 'Production'},
                {'key': 'Department', 'value': 'Finance'}
            ],
            clientRequestToken=client_request_token
        )
        
        # Try to get the guardrail ID from the response
        guardrail_id = None
        for field in ['guardrailId', 'id']:
            if field in response:
                guardrail_id = response[field]
                break
        
        print(f"Successfully created guardrail with ID: {guardrail_id}")
        print(f"Guardrail ARN: {response.get('guardrailArn')}")
        print(f"Version: {response.get('version')}")
        
        return guardrail_id
        
    except botocore.exceptions.ClientError as e:
        print(f"Error creating guardrail: {e}")
        # Check if it's because the guardrail already exists
        if 'ConflictException' in str(e) and 'already exists' in str(e):
            print("Guardrail with this name already exists. Please check all existing guardrails.")
            # Since we couldn't find it earlier but it exists, list all guardrails again
            try:
                response = bedrock.list_guardrails()
                if 'guardrailSummaries' in response:
                    print("Existing guardrails:")
                    for guardrail in response['guardrailSummaries']:
                        print(f"Name: {guardrail.get('name')}, ID: {guardrail.get('id')}")
            except:
                pass
        return None

# Execute the function if this script is run directly
if __name__ == "__main__":
    guardrail_id = create_guardrail_if_not_exists()
    print(f"Final guardrail ID: {guardrail_id}")

## 4. Save Guardrail ID to Configuration File

In [ ]:
# Update the variables dictionary with the new Guardrail ID
variables["guardrail_id"] = guardrail_id

# Save updated variables to a JSON file
with open("../variables.json", "w") as f:
    json.dump(variables, f, indent=4, default=str) 

## 5. Create a Published Version of the Guardrail

In [ ]:
# First create a published version
version_response = bedrock.create_guardrail_version(
    #guardrailIdentifier=response['guardrailId'],
    guardrailIdentifier=guardrail_id,
    description="Production version 1.0"
)



## 6. Retrieve and Save Guardrail Version

In [ ]:
guardrail_version=version_response['version']
guardrail_version


## 7. Save Guardrail Version to Configuration File

In [ ]:
# Update the variables dictionary with the new Guardrail Version
variables["guardrail_version"] = guardrail_version

# Save updated variables to a JSON file
with open("../variables.json", "w") as f:
    json.dump(variables, f, indent=4, default=str) 

> **Note**: After creating the guardrail, we recommend exploring its settings in the Amazon Bedrock console. The console provides a visual interface to understand how the guardrail is configured and allows you to test it interactively.
> 
> **[➡️ View your Guardrails in the AWS Console](https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/guardrails)**
>
> In the console, you can:
> - See all configured policies (topic filters, content filters, sensitive information policies)
> - Test sample prompts against your guardrail
> - View guardrail versions and their differences
> - Monitor usage metrics and blocked content statistics
> - Make adjustments to policies as needed